In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [30]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f"""
SELECT [SEASON_YEAR]
      ,[PLAYER_ID]
      ,[PLAYER_NAME]
      ,[NICKNAME]
      ,[TEAM_ID]
      ,[TEAM_ABBREVIATION]
      ,[TEAM_NAME]
      ,[GAME_ID]
      ,[GAME_DATE]
      ,avg(MIN) over (partition by PLAYER_ID order by GAME_DATE ROWS BETWEEN 82 PRECEDING AND CURRENT ROW) as Minutes_Per_Game
   
  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  where yearSeason =2024
"""
df = pd.read_sql(sql,cnxn)

C:\Users\bobby\AppData\Local\Temp\ipykernel_15328\506736713.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,cnxn)


In [33]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f"""
SELECT [SEASON_YEAR]
      ,[PLAYER_ID]
      ,[PLAYER_NAME]
      ,[NICKNAME]
      ,[TEAM_ID]
      ,[TEAM_ABBREVIATION]
      ,[TEAM_NAME]
      ,[GAME_ID]
      ,[GAME_DATE]
      ,avg(MIN) over (partition by PLAYER_ID order by GAME_DATE ROWS BETWEEN 82 PRECEDING AND CURRENT ROW) as Minutes_Per_Game
   
  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  where yearSeason =2024
"""
df = pd.read_sql(sql,cnxn)

# Convert GAME_DATE to datetime and sort
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
df = df.sort_values(by=['GAME_DATE'])

out_players_records = []

def was_player_traded(player_id, team_id, game_date):
    last_team_played_for = df[(df['PLAYER_ID'] == player_id) & (df['GAME_DATE'] < game_date)].tail(1)['TEAM_ID'].values
    return not last_team_played_for or (last_team_played_for[0] != team_id)

for game_id in df['GAME_ID'].unique():
    game_df = df[df['GAME_ID'] == game_id]
    teams_in_game = game_df['TEAM_ID'].unique()
    
    for team_id in teams_in_game:
        players_in_game = set(game_df[game_df['TEAM_ID'] == team_id]['PLAYER_ID'])
        previous_games = df[(df['TEAM_ID'] == team_id) & (df['GAME_DATE'] < game_df['GAME_DATE'].min())]
        expected_players = set(previous_games['PLAYER_ID'])
        
        out_players = []
        for player_id in expected_players:
            if player_id not in players_in_game and was_player_traded(player_id, team_id, game_df['GAME_DATE'].min()) == False:
                out_players.append(player_id)
        
        for pid in out_players:
            player_last_game = df[(df['PLAYER_ID'] == pid) & (df['GAME_DATE'] < game_df['GAME_DATE'].min())].tail(1)
            minutes_per_game = player_last_game['Minutes_Per_Game'].iloc[0] if not player_last_game.empty else 0
            player_name = df[df['PLAYER_ID'] == pid]['PLAYER_NAME'].iloc[0]
            
            out_players_records.append({
                'GAME_ID': game_id,
                'TEAM_ID': team_id,
                'OUT_PLAYER_ID': pid,
                'PLAYER_NAME': player_name,
                'Minutes_Per_Game': minutes_per_game
            })

out_players_df = pd.DataFrame(out_players_records)

# Calculating the counts
counts_df = out_players_df.groupby(['GAME_ID', 'TEAM_ID']).apply(
    lambda x: pd.Series({
        'number_of_players_who_over_avg_over_15_min_out': (x['Minutes_Per_Game'] > 15).sum(),
        'number_of_players_who_over_avg_over_20_min_out': (x['Minutes_Per_Game'] > 20).sum(),
        'number_of_players_who_over_avg_over_25_min_out': (x['Minutes_Per_Game'] > 25).sum()
    })
).reset_index()

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

# Clear the table only if you're sure you want to remove existing data
cursor.execute("DELETE FROM player_injury_report")
cnxn.commit()

# Check if there are records to insert in the 'counts_df' DataFrame
if not counts_df.empty:
    insert_stmt = '''
    INSERT INTO player_injury_report (TEAM_ID, GAME_ID, number_of_players_who_over_avg_over_15_min_out, number_of_players_who_over_avg_over_20_min_out, number_of_players_who_over_avg_over_25_min_out)
    VALUES (?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in counts_df.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Ensure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
            int(row['TEAM_ID']),  # Convert TEAM_ID to Python int
            int(row['GAME_ID']),  # Convert GAME_ID to Python int
            int(row['number_of_players_who_over_avg_over_15_min_out']),
            int(row['number_of_players_who_over_avg_over_20_min_out']),
            int(row['number_of_players_who_over_avg_over_25_min_out'])
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Depending on your error handling preference, you might choose to break or continue

    cursor.close()
    cnxn.close()
else:
    print("No data to insert.")


,GAME_ID,TEAM_ID,OUT_PLAYER_ID,PLAYER_NAME,Minutes_Per_Game
0,22300076,1610612747,1631108,Max Christie,1.250000
1,22300076,1610612747,1641721,Maxwell Lewis,1.250000
2,22300076,1610612756,1626164,Devin Booker,36.966667
3,22300082,1610612741,1641763,Julian Phillips,3.666667
4,22300082,1610612741,1631207,Dalen Terry,3.666667
...,...,...,...,...,...
15476,22301160,1610612751,1628372,Dennis Smith Jr.,18.904613
15477,22301160,1610612751,1627732,Ben Simmons,23.927778
15478,22301160,1610612751,1630553,Keon Johnson,12.762917
15479,22301160,1610612751,1629022,Lonnie Walker IV,16.946250


In [46]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

# Clear the table only if you're sure you want to remove existing data
cursor.execute("DELETE FROM player_injury_report")
cnxn.commit()

# Check if there are records to insert in the 'counts_df' DataFrame
if not counts_df.empty:
    insert_stmt = '''
    INSERT INTO player_injury_report (TEAM_ID, GAME_ID, number_of_players_who_over_avg_over_15_min_out, number_of_players_who_over_avg_over_20_min_out, number_of_players_who_over_avg_over_25_min_out)
    VALUES (?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in counts_df.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Ensure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
            int(row['TEAM_ID']),  # Convert TEAM_ID to Python int
            int(row['GAME_ID']),  # Convert GAME_ID to Python int
            int(row['number_of_players_who_over_avg_over_15_min_out']),
            int(row['number_of_players_who_over_avg_over_20_min_out']),
            int(row['number_of_players_who_over_avg_over_25_min_out'])
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Depending on your error handling preference, you might choose to break or continue

    cursor.close()
    cnxn.close()
else:
    print("No data to insert.")


In [44]:
# out_players_df.loc[out_players_df['GAME_ID']==22301229]

In [43]:
# Calculating the counts
counts_df = out_players_df.groupby(['GAME_ID', 'TEAM_ID']).apply(
    lambda x: pd.Series({
        'number_of_players_who_over_avg_over_15_min_out': (x['Minutes_Per_Game'] > 15).sum(),
        'number_of_players_who_over_avg_over_20_min_out': (x['Minutes_Per_Game'] > 20).sum(),
        'number_of_players_who_over_avg_over_25_min_out': (x['Minutes_Per_Game'] > 25).sum()
    })
).reset_index()

counts_df.columns

Index(['GAME_ID', 'TEAM_ID', 'number_of_players_who_over_avg_over_15_min_out',
       'number_of_players_who_over_avg_over_20_min_out',
       'number_of_players_who_over_avg_over_25_min_out'],
      dtype='object')

In [36]:
# test = out_players_df.loc[out_players_df['TEAM_ID']==1610612737]
out_players_df.groupby(by='GAME_ID')['OUT_PLAYER_ID'].count()

GAME_ID
22300001    11
22300002    11
22300003    11
22300004     9
22300005    12
            ..
22301226    15
22301227    11
22301228     9
22301229    12
22301230     9
Name: OUT_PLAYER_ID, Length: 1177, dtype: int64

In [10]:
out_players_df

,GAME_ID,TEAM_ID,OUT_PLAYERS
0,22300076,1610612747,"[Maxwell Lewis, Max Christie]"
1,22300076,1610612756,[Devin Booker]
2,22300077,1610612766,[Nick Smith Jr.]
3,22300083,1610612745,"[Nate Williams, Cam Whitmore, Jock Landale, Je..."
4,22300086,1610612753,"[Chuma Okeke, Anthony Black, Caleb Houstan, Go..."
...,...,...,...
2320,22301145,1610612755,"[Kyle Lowry, Tyrese Maxey, Jaden Springer, Mar..."
2321,22301152,1610612750,"[Luka Garza, Shake Milton, Troy Brown Jr., Kar..."
2322,22301152,1610612764,"[Mike Muscala, Tyus Jones, Danilo Gallinari, B..."
2323,22301144,1610612766,"[Seth Curry, Theo Maledon, Terry Rozier, Mark ..."


In [7]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f"""
SELECT [gameId]
	  , lgl.GAME_DATE
      ,[teamId]
      ,[teamCity]
      ,[teamName]
      ,[teamTricode]
      ,[teamSlug]
      ,[minutes]
      ,[estimatedOffensiveRating]
      ,[offensiveRating]
      ,[estimatedDefensiveRating]
      ,[defensiveRating]
      ,[estimatedNetRating]
      ,[netRating]
      ,[assistPercentage]
      ,[assistToTurnover]
      ,[assistRatio]
      ,[offensiveReboundPercentage]
      ,[defensiveReboundPercentage]
      ,[reboundPercentage]
      ,[estimatedTeamTurnoverPercentage]
      ,[turnoverRatio]
      ,[effectiveFieldGoalPercentage]
      ,[trueShootingPercentage]
      ,[usagePercentage]
      ,[estimatedUsagePercentage]
      ,[estimatedPace]
      ,[pace]
      ,[pacePer40]
      ,[possessions]
      ,[PIE]
  FROM [nba_game_data].[dbo].[TeamBoxScoreAdvancedV3] tbsa
    left outer join [nba_game_data].[dbo].[LeagueGameLog] lgl
  on 
  tbsa.teamId = lgl.TEAM_ID
  and tbsa.gameId = lgl.GAME_ID
where lgl.yearSeason = 2024
"""
df = pd.read_sql(sql,cnxn)
# df

C:\Users\bobby\AppData\Local\Temp\ipykernel_10636\1343648910.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,cnxn)


In [8]:
df

,gameId,GAME_DATE,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,estimatedOffensiveRating,offensiveRating,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,0022300956,2024-03-14,1610612741,Chicago,Bulls,CHI,bulls,240:00,116.8,119.4,...,14.0,0.570,0.597,1.0,0.198,92.60,92.0,76.67,93.0,0.419
1,0022300956,2024-03-14,1610612746,LA,Clippers,LAC,clippers,240:00,139.8,138.5,...,11.0,0.725,0.731,1.0,0.200,92.60,92.0,76.67,91.0,0.581
2,0022300957,2024-03-14,1610612745,Houston,Rockets,HOU,rockets,240:00,124.4,123.9,...,17.4,0.655,0.692,1.0,0.197,108.46,108.5,90.42,109.0,0.559
3,0022300957,2024-03-14,1610612764,Washington,Wizards,WAS,wizards,240:00,109.8,110.2,...,13.0,0.559,0.587,1.0,0.198,108.46,108.5,90.42,108.0,0.441
4,0022300955,2024-03-14,1610612738,Boston,Celtics,BOS,celtics,240:00,129.8,132.3,...,10.4,0.669,0.677,1.0,0.195,98.12,96.5,80.42,96.0,0.562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0022300745,2024-02-09,1610612766,Charlotte,Hornets,CHA,hornets,240:00,85.1,84.8,...,15.2,0.386,0.458,1.0,0.199,100.16,99.0,82.50,99.0,0.321
2206,0022300742,2024-02-09,1610612755,Philadelphia,76ers,PHI,sixers,240:00,117.8,121.0,...,10.0,0.505,0.546,1.0,0.199,101.14,99.5,82.92,100.0,0.439
2207,0022300742,2024-02-09,1610612737,Atlanta,Hawks,ATL,hawks,240:00,127.6,128.3,...,13.1,0.617,0.665,1.0,0.201,101.14,99.5,82.92,99.0,0.561
2208,0022300747,2024-02-09,1610612747,Los Angeles,Lakers,LAL,lakers,240:00,132.3,137.6,...,7.9,0.636,0.681,1.0,0.196,103.60,101.0,84.17,101.0,0.547


In [15]:
import pandas as pd

# Assuming df is your DataFrame loaded with the initial data

# Convert the 'GAME_DATE' column to datetime if not already
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Ensure the data is sorted
df.sort_values(by=['teamId', 'GAME_DATE'], inplace=True)

# Create a unique list of all teams and all dates
unique_teams = df['teamId'].unique()
unique_dates = pd.date_range(start=df['GAME_DATE'].min(), end=df['GAME_DATE'].max())

# Create a DataFrame from the product of unique_dates and unique_teams
from itertools import product
all_combinations = pd.DataFrame(product(unique_dates, unique_teams), columns=['GAME_DATE', 'teamId'])

# Calculate the running average for defensiveRating, excluding the current game
df['cumulativeDR'] = df.groupby('teamId')['defensiveRating'].apply(lambda x: x.shift(1).expanding().mean())

# Merge the running average back onto the all_combinations DataFrame
complete_df = pd.merge(all_combinations, df[['GAME_DATE', 'teamId', 'cumulativeDR']], on=['GAME_DATE', 'teamId'], how='left')

# Forward fill the missing values for each team to carry the last known average forward
complete_df['cumulativeDR'] = complete_df.groupby('teamId')['cumulativeDR'].ffill()

# First, ensure the DataFrame is sorted by date and then by the defensive rating in ascending order 
# (since a lower defensive rating is better in basketball).
complete_df.sort_values(by=['GAME_DATE', 'cumulativeDR'], ascending=[True, True], inplace=True)

# Rank the teams within each day
complete_df['daily_rank'] = complete_df.groupby('GAME_DATE')['cumulativeDR'].rank(method='min')


C:\Users\bobby\AppData\Local\Temp\ipykernel_10636\3471736563.py:20: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['cumulativeDR'] = df.groupby('teamId')['defensiveRating'].apply(lambda x: x.shift(1).expanding().mean())


In [20]:
complete_df.dropna()#.loc[complete_df['GAME_DATE']=='2024-01-29']

,GAME_DATE,teamId,cumulativeDR,daily_rank
79,2023-10-26,1610612756,102.000000,1.0
70,2023-10-26,1610612747,125.300000,2.0
106,2023-10-27,1610612753,90.500000,1.0
114,2023-10-27,1610612761,93.100000,2.0
109,2023-10-27,1610612756,102.000000,3.0
...,...,...,...,...
4757,2024-03-30,1610612754,118.094595,26.0
4740,2024-03-30,1610612737,118.487671,27.0
4768,2024-03-30,1610612765,118.673973,28.0
4769,2024-03-30,1610612766,119.038889,29.0
